In [ ]:
"""
Imports
"""
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils.memory import check_memory, clear_all_cuda_memory
from utils.store_topk import convert_topk_to_df
from utils.vocab import export_vocab_as_csv

main_device = 'cuda:0'
seed = 1234
clear_all_cuda_memory()
check_memory()

## Load base model

In [ ]:
"""
Load the base tokenizer/model
"""
model_id = [
    'allenai/OLMoE-1B-7B-0125-Instruct',
    'Qwen/Qwen1.5-MoE-A2.7B-Chat',
    'deepseek-ai/DeepSeek-V2-Lite'
][0]

model_prefix = {
    'allenai/OLMoE-1B-7B-0125-Instruct': 'olmoe',
    'Qwen/Qwen1.5-MoE-A2.7B-Chat': 'qwenv15',
    'deepseek-ai/DeepSeek-V2-Lite': 'dsv2',

}[model_id]

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token = False, add_bos_token = False, padding_side = 'left')
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.bfloat16, trust_remote_code = True).cuda().eval()

## Get dataset

In [ ]:
ds = load_dataset("allenai/c4", 'en', split = 'validation', streaming = True).shuffle(seed = 42, buffer_size = 5_000_000)
ds_iter = iter(ds)

c4_raw = []
for _ in range(0, 5_000_000):
    sample = next(ds_iter, None)
    if sample is None:
        break
    c4_raw.append(sample['text'])

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, tokenizer_output):
        self.input_ids = tokenizer_output['input_ids']
        self.attention_mask = tokenizer_output['attention_mask']
        self.tokens = tokenizer.batch_decode(self.input_ids)

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {
            'tokens': self.tokens[idx],
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

res = tokenizer(c4_raw, add_special_tokens = False, max_length = 512, padding = 'max_length', truncation = True, return_tensors = 'pt')
c4_dl = DataLoader(TextDataset(res), batch_size = 16, shuffle = False)

## Get expert selections + export

In [ ]:
"""
Define model-specific functions

These functions must return a dict with keys:
- `logits`: The standard B x N x V LM output
- `all_topk_experts`: A list of length equal to the number of MoE layers, with each element a BN x topk tensor of expert IDs
- `all_topkweights`: A list of length equal to the number of MoE layers, with each element a BN x topk tensor of expert weights
"""

@torch.no_grad()
def run_olmoe_return_topk(input_ids, attention_mask):
    input_embeds = model.model.embed_tokens(input_ids)
    
    cache_position = torch.arange(0, input_embeds.shape[1], device = input_embeds.device)
    position_ids = cache_position.unsqueeze(0)
    causal_mask = model.model._update_causal_mask(attention_mask, input_embeds, cache_position, None, None)

    hidden_state = input_embeds
    position_embeddings = model.model.rotary_emb(hidden_state, position_ids)

    all_topk_experts = []
    all_topk_weights = []
    for layer in model.model.layers:
        # SA
        residual = hidden_state
        hidden_state = layer.input_layernorm(hidden_state)
        hidden_state, _, _ = layer.self_attn(hidden_states = hidden_state, attention_mask = causal_mask, position_ids = position_ids, position_embeddings = position_embeddings)
        hidden_state = residual + hidden_state
        residual = hidden_state
        hidden_state = layer.post_attention_layernorm(hidden_state)

        ####### OlMoESparseMoeBlock - below code replaces hidden_state = layer.mlp(hidden_state)
        batch_size, sequence_length, hidden_dim = hidden_state.shape
        moe_hidden_state = hidden_state.view(-1, hidden_dim)
        router_logits = layer.mlp.gate(moe_hidden_state)

        routing_weights = torch.nn.functional.softmax(router_logits, dim=1, dtype=torch.float)
        routing_weights, selected_experts = torch.topk(routing_weights, layer.mlp.top_k, dim=-1, sorted = True)
        routing_weights = routing_weights.to(moe_hidden_state.dtype)
        final_hidden_states = torch.zeros((batch_size * sequence_length, hidden_dim), dtype = hidden_state.dtype, device = hidden_state.device)
        expert_mask = torch.nn.functional.one_hot(selected_experts, num_classes = layer.mlp.num_experts).permute(2, 1, 0)

        # Loop over all available experts in the model and perform the computation on each expert
        for expert_idx in range(layer.mlp.num_experts):
            expert_layer = layer.mlp.experts[expert_idx]
            idx, top_x = torch.where(expert_mask[expert_idx])

            current_state = moe_hidden_state[None, top_x].reshape(-1, hidden_dim)
            current_hidden_states = expert_layer(current_state) * routing_weights[top_x, idx, None]

            final_hidden_states.index_add_(0, top_x, current_hidden_states.to(moe_hidden_state.dtype))

        final_hidden_states = final_hidden_states.reshape(batch_size, sequence_length, hidden_dim)
        #######

        hidden_state = final_hidden_states
        hidden_state = residual + hidden_state

        all_topk_experts.append(selected_experts.detach().cpu())
        all_topk_weights.append(routing_weights.detach().cpu().to(torch.float32))

    hidden_state = model.model.norm(hidden_state)
    logits = model.lm_head(hidden_state)
    return {'logits': logits, 'all_topk_experts': all_topk_experts, 'all_topk_weights': all_topk_weights}

@torch.no_grad()
def run_qwenv15_return_topk(input_ids, attention_mask):
    input_embeds = model.model.embed_tokens(input_ids)
    
    cache_position = torch.arange(0, input_embeds.shape[1], device = input_embeds.device)
    position_ids = cache_position.unsqueeze(0)
    causal_mask = model.model._update_causal_mask(attention_mask, input_embeds, cache_position, None, None)

    hidden_state = input_embeds
    position_embeddings = model.model.rotary_emb(hidden_state, position_ids)

    all_topk_experts = []
    all_topk_weights = []
    for layer in model.model.layers:
        # SA
        residual = hidden_state
        hidden_state = layer.input_layernorm(hidden_state)
        hidden_state, _, _ = layer.self_attn(hidden_states = hidden_state, attention_mask = causal_mask, position_ids = position_ids, position_embeddings = position_embeddings)
        hidden_state = residual + hidden_state
        residual = hidden_state
        hidden_state = layer.post_attention_layernorm(hidden_state)

        ####### Qwen2MoeSparseMoeBlock - below code replaces hidden_state = layer.mlp(hidden_state)
        batch_size, sequence_length, hidden_dim = hidden_state.shape
        moe_hidden_state = hidden_state.view(-1, hidden_dim)
        router_logits = layer.mlp.gate(moe_hidden_state) # Size (BN, n_experts)

        routing_weights = torch.nn.functional.softmax(router_logits, dim = 1, dtype = torch.float)
        routing_weights, selected_experts = torch.topk(routing_weights, layer.mlp.top_k, dim = -1, sorted = True)
        routing_weights = routing_weights.to(moe_hidden_state.dtype)

        final_hidden_states = torch.zeros((batch_size * sequence_length, hidden_dim), dtype = moe_hidden_state.dtype, device = moe_hidden_state.device)

        # One hot encode the selected experts to create an expert mask 
        expert_mask = torch.nn.functional.one_hot(selected_experts, num_classes = layer.mlp.num_experts).permute(2, 1, 0)

        # Loop over all available experts in the model and perform the computation on each expert
        for expert_idx in range(layer.mlp.num_experts):
            expert_layer = layer.mlp.experts[expert_idx]
            idx, top_x = torch.where(expert_mask[expert_idx])
            # Index the correct hidden states and compute the expert hidden state for the current expert.
            current_state = moe_hidden_state[None, top_x].reshape(-1, hidden_dim)
            current_hidden_states = expert_layer(current_state) * routing_weights[top_x, idx, None]
            # However `index_add_` only support torch tensors for indexing so we'll use the `top_x` tensor here.
            final_hidden_states.index_add_(0, top_x, current_hidden_states.to(moe_hidden_state.dtype))

        shared_expert_output = layer.mlp.shared_expert(moe_hidden_state)
        shared_expert_output = torch.nn.functional.sigmoid(layer.mlp.shared_expert_gate(moe_hidden_state)) * shared_expert_output

        final_hidden_states = (final_hidden_states + shared_expert_output).reshape(batch_size, sequence_length, hidden_dim)
        #######
        hidden_state = final_hidden_states
        hidden_state = residual + hidden_state

        all_topk_experts.append(selected_experts.detach().cpu())
        all_topk_weights.append(routing_weights.detach().cpu().to(torch.float32))

    hidden_state = model.model.norm(hidden_state)
    logits = model.lm_head(hidden_state)
    return {'logits': logits, 'all_topk_experts': all_topk_experts, 'all_topk_weights': all_topk_weights}


from transformers.modeling_attn_mask_utils import _prepare_4d_causal_attention_mask
@torch.no_grad()
def run_dsv2_return_topk(input_ids, attention_mask):
    B, N = input_ids.shape[:2]
    position_ids = torch.arange(0, N, dtype=torch.long, device = main_device).unsqueeze(0)

    inputs_embeds = model.model.embed_tokens(input_ids)
    attention_mask = _prepare_4d_causal_attention_mask(attention_mask, (B, N), inputs_embeds, 0,)

    hidden_state = inputs_embeds
    all_topk_experts = []
    all_topk_weights = []
    for layer_ix, decoder_layer in enumerate(model.model.layers):
        # layer_outputs = decoder_layer(hidden_state, attention_mask = attention_mask, position_ids = position_ids,)
        residual = hidden_state
        hidden_state = decoder_layer.input_layernorm(hidden_state)
        # Self Attention
        hidden_state, self_attn_weights, present_key_value = decoder_layer.self_attn(hidden_states = hidden_state, attention_mask = attention_mask, position_ids = position_ids)
        hidden_state = residual + hidden_state
        # Fully Connected
        residual = hidden_state
        hidden_state = decoder_layer.post_attention_layernorm(hidden_state)
        ## MLP
        if 'DeepseekV2MLP' in str(type(decoder_layer.mlp)):
            hidden_state = decoder_layer.mlp(hidden_state)
        else:
            identity = hidden_state
            orig_shape = hidden_state.shape
            ### moegate - originally topk_idx, topk_weight, aux_loss = decoder_layer.mlp.gate(hidden_state)
            bsz, seq_len, h = hidden_state.shape
            moe_hidden_state = hidden_state.view(-1, h)
            logits = torch.nn.functional.linear(moe_hidden_state.type(torch.float32), decoder_layer.mlp.gate.weight.type(torch.float32), None)
            scores = logits.softmax(dim=-1, dtype=torch.float32)
            topk_weight, topk_idx = torch.topk(scores, k=decoder_layer.mlp.gate.top_k, dim=-1, sorted=False)
            topk_weight = topk_weight * decoder_layer.mlp.gate.routed_scaling_factor
            ####
            hidden_state = hidden_state.view(-1, hidden_state.shape[-1])
            ### moe infer
            x = hidden_state
            topk_ids = topk_idx
            cnts = topk_ids.new_zeros((topk_ids.shape[0], len(decoder_layer.mlp.experts)))
            cnts.scatter_(1, topk_ids, 1)
            tokens_per_expert = cnts.sum(dim=0)
            idxs = topk_ids.view(-1).argsort()
            sorted_tokens = x[idxs // topk_ids.shape[1]]
            tokens_per_expert = tokens_per_expert.cpu().numpy()
            outputs = []
            start_idx = 0
            for i, num_tokens in enumerate(tokens_per_expert):
                end_idx = start_idx + num_tokens
                if num_tokens == 0:
                    continue
                expert = decoder_layer.mlp.experts[i + decoder_layer.mlp.ep_rank * decoder_layer.mlp.experts_per_rank]
                tokens_for_this_expert = sorted_tokens[start_idx:end_idx]
                expert_out = expert(tokens_for_this_expert)
                outputs.append(expert_out)
                start_idx = end_idx
            outs = torch.cat(outputs, dim=0) if len(outputs) else sorted_tokens.new_empty(0)
            new_x = torch.empty_like(outs)
            new_x[idxs] = outs
            final_out = (new_x.view(*topk_ids.shape, -1).type(topk_weight.dtype).mul_(topk_weight.unsqueeze(dim=-1)).sum(dim=1).type(new_x.dtype))
            ###
            y = final_out.view(*orig_shape)
            if decoder_layer.mlp.config.n_shared_experts is not None:
                y = y + decoder_layer.mlp.shared_experts(identity)
            hidden_state = y

            all_topk_experts.append(topk_ids)
            all_topk_weights.append(topk_weight)

        hidden_state = residual + hidden_state

    hidden_state = model.model.norm(hidden_state)
    logits = model.lm_head(hidden_state)
    return {'logits': logits, 'all_topk_experts': all_topk_experts, 'all_topk_weights': all_topk_weights}
    


In [ ]:
""" 
Export expert selections
"""
def run_and_export_topk(model_id, model_prefix, c4_dl, batches_to_test = 250):
    """
    Run forward passes on a given model ID, return topk df
    """
    b_count = 0

    for batch_ix, batch in tqdm(enumerate(c4_dl)):

        input_ids = batch['input_ids'].to(main_device)
        attention_mask = batch['attention_mask'].to(main_device)
        
        if model_prefix == 'olmoe':
            fn = run_olmoe_return_topk
        elif model_prefix == 'qwenv15':
            fn = run_qwenv15_return_topk
        elif model_prefix == 'dsv2':
            fn = run_dsv2_return_topk
        else:
            raise Exception('Unsupported model!')

        output = fn(input_ids, attention_mask)

        topk_df = convert_topk_to_df(input_ids, output['all_topk_experts'], output['all_topk_weights'])
        topk_df =\
            topk_df[topk_df['token_id'] != tokenizer.pad_token_id]\
            .assign(
                batch_ix = batch_ix,
                weight = lambda df: df['weight'].round(3)
            )

        topk_df.to_csv(
            f'{model_prefix}-c4-routes.csv',
            mode = 'w' if batch_ix == 0 else 'a',
            index = False,
            header = (batch_ix == 0)
        )

        topk_df[topk_df['topk_ix'] == 1].to_csv(
            f'{model_prefix}-c4-routes-top1.csv',
            mode = 'w' if batch_ix == 0 else 'a',
            index = False,
            header = (batch_ix == 0)
        )

        b_count += 1
        if b_count >= batches_to_test:
            break

    return True

export_vocab_as_csv(tokenizer, f'{model_prefix}-vocab.csv')
run_and_export_topk(model_id, model_prefix, c4_dl, batches_to_test = 250)